In [0]:
# solo en bronce ,ya que en pipeline se hace con parametro general
dbutils.widgets.text("path", "", "")
path = dbutils.widgets.get("path")

In [0]:
from pyspark.sql.functions import *

df_bankloans_gold = spark.read.format("delta").load(f"/{path}/plata/delta_tables")


# Ingresos promedio por educación
df_ingreso_educacion = df_bankloans_gold.groupBy("Education") \
    .agg(avg("income").alias("avg_income")) \
    .orderBy("avg_income", ascending=False)

# result = spark.sql(
#     """
#     SELECT Education, AVG(Income) AS avg_income_by_education FROM bank_loans
#     GROUP BY Education
#     ORDER BY AVG(Income) DESC
#     """
# )


display(df_ingreso_educacion)

df_ingreso_educacion.write.format("delta").mode("overwrite").option("mergeSchema", "true").save(f"/{path}/oro/avg_income_by_education")



In [0]:
# Edad promedio por nivel de educación
df_age_educacion = df_bankloans_gold.groupBy("Education") \
    .agg(avg("Age").alias("Avg_age")) \
    .orderBy("avg_age", ascending=False)

result= spark.sql(
    """
    SELECT Education, AVG(Age) AS Avg_age FROM loans
    GROUP BY Education
    ORDER BY AVG(Age) DESC
    """
)
display(result)

display(df_age_educacion)

df_age_educacion.write.format("delta").mode("overwrite").option("mergeSchema", "true").save(f"/{path}/oro/avg_age_by_education")

In [0]:
# Ingreso total y promedio por número de hijos (Family)

avg_ingresos_familia = df_bankloans_gold.groupBy("Family") \
    .agg(
        sum("income").alias("Total_income"),
        avg("income").alias("Avg_income")
    ).orderBy("Family")

display(avg_ingresos_familia)

avg_ingresos_familia.write.format("delta").mode("overwrite").option("mergeSchema", "true").mode("overwrite").save(f"/{path}/oro/avg_ingresos_familia")

# avg_ingresos_familia = spark.sql("""
#     SELECT Family, SUM(Income) as total_income, AVG(Income) AS Avg_income FROM loans
#     GROUP BY Family
#     ORDER BY SUM(Income) DESC      
# """)
# display(avg_ingresos_familia)




In [0]:
# Ingreso por hipoteca

from pyspark.sql.window import Window
from pyspark.sql.functions import col, when, count, sum

df_mortgage_group = df_bankloans_gold.withColumn(
    "Has_mortgage",
    when(col("Mortgage") > 0, "Yes").otherwise("No")
).groupBy("Has_mortgage") \
 .agg(count("*").alias("Total")) \
.withColumn("Percentage", col("Total") / sum("Total").over(Window.rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)) * 100)

# result = spark.sql(
#     """
#     SELECT 
#         CASE WHEN Mortgage > 0 THEN 'Yes' ELSE 'No' END AS has_mortgage, 
#         COUNT(*) AS total, 
#         COUNT(*) * 100.0 / SUM(COUNT(*)) OVER () AS percentage 
#     FROM loans
#     GROUP BY has_mortgage
#     ORDER BY total DESC
#     """
# )
# display(result)
display(df_mortgage_group)

df_mortgage_group.write.format("delta").mode("overwrite").option("mergeSchema", "true").save(f"/{path}/oro/mortgage_group")


In [0]:
df_income_segmentado = df_bankloans_gold.withColumn(
    "Segmento_ingreso",
    when(col("Income") < 50, "<50k")
    .when((col("Income") >= 50) & (col("Income") < 100), "50k-100k")
    .when((col("Income") >= 100) & (col("Income") < 150), "100k-150k")
    .otherwise("150k+")
)

df_aprobacion_por_segmento = df_income_segmentado.groupBy("Segmento_ingreso") \
    .agg(
        count("*").alias("Total_clientes"),
        sum("Personal_Loan").alias("Prestamos_aprobados")
    ) \
    .withColumn("Tasa_aprobacion", col("Prestamos_aprobados") / col("Total_clientes") * 100) \
    .orderBy("Segmento_ingreso")

display(df_aprobacion_por_segmento)

df_aprobacion_por_segmento.write.format("delta").mode("overwrite").option("mergeSchema", "true").save(f"/{path}/oro/tasa_aprobacion_por_segmento_ingreso")